In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import log_reg_tensorflow as lrf
import matplotlib.pyplot as plt

/home/caio/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/caio/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/caio/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/caio/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
df = pd.read_csv("mnist_dataset/train.csv")

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df[['pixel{}'.format(i) for i in range(784)]].values.astype(np.float32)
mu = X.mean(axis=0)
std = X.std(axis=0)
np.place(std, std == 0, 1)
X_train, X_test, y_train, y_test = train_test_split((X-mu)/std, 
                                                    df['label'].values.astype(np.int32), test_size=0.2, random_state=42)

In [ ]:
imageside = 28
cols = 4
rows = 6
f, axarr = plt.subplots(rows,cols,figsize=(18,10))
for i in range(rows):
    for j in range(cols):
        axarr[i,j].imshow(X_train[cols*i+j].reshape(imageside,imageside), cmap='gray')
        axarr[i,j].set_title(y_train[cols*i+j])

f.tight_layout(pad=1.5)

In [ ]:
import importlib
importlib.reload(lrf)

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

In [ ]:
logreg = lrf.MulticlassLogisticRegression(num_epochs=1000, print_epoch=10, learning_rate=2e-4, gd_type='full')
logreg.fit(X_train.astype(np.float32), y_train.astype(np.float32))

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
print("Model accuracy on training set: {}\n".format(round(logreg.score(y_train, logreg.predict_proba(X_train)),4))+
      "Model accuracy on test set: {}".format(logreg.score(y_test, logreg.predict_proba(X_test))))

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle

class MulticlassLogisticRegression:
    """
    Multiclass logreg scikit-learn-like with softmax.

    Author:
        Caio Martins
    """

    def __init__(self, learning_rate=1e-3, num_epochs=10000, verbose=True, print_epoch=1000, eps=1e-5,
                 gd_type='full', batch_number = 10, mu = 0):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.verbose = verbose
        self.print_epoch = print_epoch
        self.eps = eps
        self.gd_type = gd_type
        self.batch_number = batch_number
        self.mu = mu

    def score(self, y, y_pred):
        accuracy = (y == y_pred).sum()/y.shape[0]
        return accuracy

    def _get_dims(self, X, y):
        self.K = np.unique(y).size
        self.N, self.D = X.shape

    def _initialize_weights(self):
        self.W = np.random.randn(self.D,self.K) / np.sqrt(self.D)
        self.b = np.zeros(self.K)

    def fit(self, X, y=None):
        self._get_dims(X, y)
        self._initialize_weights()
        self.X = X.reshape((-1,self.D))
        self.y = y

        self.T = np.array([[0. if y[j] != i else 1. for i in range(self.K)] for j in range(self.N)])\
                    .astype(np.float32)\
                    .reshape((-1,self.K))

        inputs = tf.placeholder(tf.float32, [None,self.D])
        labels = tf.placeholder(tf.float32, [None,self.K])
        print(self.W)
        W = tf.Variable(self.W.astype(np.float32))
        b = tf.Variable(self.b.astype(np.float32))

        Z = tf.matmul(inputs, W) + b #logits

        cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z, labels=labels))

        train_op = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(cost)
        self.pred_op = tf.arg_max(Z, 1)


        self.costs = []
        self.clf_rates = []

        if self.gd_type == 'full':
    
            init = tf.global_variables_initializer()    
            with tf.Session() as session:

                session.run(init)

                for epoch in range(self.num_epochs):
                    print("epoch: %d" % epoch, X.shape)
                    session.run(train_op, feed_dict = {inputs: self.X, labels: self.T})
#                     y_pred = session.run(self.pred_op, feed_dict = {inputs: X})

                    self.costs.append(cost.eval())
                    self.clf_rates.append(self.score(y, y_pred))
                    
                    if epoch % self.print_epoch == 0:
                        if self.verbose:
                            print("epoch: ", epoch,
                                  "cost: ", self.costs[-1],
                                  "accuracy: ", self.clf_rates[-1])

In [18]:
tf.reset_default_graph()
logreg = MulticlassLogisticRegression(num_epochs=100, print_epoch=10, learning_rate=2e-4, gd_type='full')
logreg.fit(X_train, y_train.astype(np.float32))

[[ 0.00591304  0.01692833  0.01511701 ... -0.02290838 -0.00857448
   0.0144155 ]
 [-0.019797   -0.01673613  0.06426154 ...  0.01119538  0.01399511
   0.0661476 ]
 [-0.02009664 -0.0132884  -0.03876095 ... -0.0221053   0.02080401
   0.00872052]
 ...
 [ 0.05537791  0.02906808 -0.01341107 ... -0.00845089 -0.02255576
  -0.01579293]
 [-0.00917571  0.02420007  0.02347245 ...  0.06594794  0.03977725
  -0.04952736]
 [-0.00821363  0.0094276   0.03529363 ... -0.02591356  0.0209468
  -0.06701962]]
epoch: 0 (33600, 784)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,10]
	 [[node Placeholder_1 (defined at <ipython-input-16-ca35e9d3da9d>:47) ]]

Original stack trace for 'Placeholder_1':
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home

In [14]:
logreg.T.reshape((-1,10))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
def score(y, y_pred):
    accuracy = (y == y_pred).sum()/y.shape[0]
    return accuracy

def get_dims(X, y):
    K = np.unique(y).size
    N, D = X.shape
    return N, D, K

def initialize_weights(N, D, K):
    W = np.random.randn(D, K) / np.sqrt(D)
    b = np.zeros(K)
    return W, b

num_epochs=100
print_epoch=10 
learning_rate=2e-4

N, D, K = get_dims(X_train, y_train)
W, b = initialize_weights(N, D, K)

T = np.array([[0. if y_train[j] != i else 1. for i in range(K)] for j in range(N)])\
            .astype(np.float32)\
            .reshape((-1,K))

tfX = tf.placeholder(tf.float32, [None,D])
tfY = tf.placeholder(tf.float32, [None,K])

Wtf = tf.Variable(W.astype(np.float32))
btf = tf.Variable(b.astype(np.float32))

Z = tf.matmul(tfX, Wtf) + btf #logits

cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z, labels=tfY))

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
pred_op = tf.argmax(Z, 1)


costs = []
#     self.clf_rates = []

init = tf.global_variables_initializer()    
with tf.Session() as session:

    session.run(init)

    for epoch in range(num_epochs):
        print("epoch: %d" % epoch, X_train.shape)
        session.run(train_op, feed_dict = {tfX: X_train, tfY: T})
#                     y_pred = session.run(self.pred_op, feed_dict = {inputs: X})

        costs.append(cost.eval())
#             self.clf_rates.append(self.score(y, y_pred))

        if epoch % print_epoch == 0:
            print("epoch: ", epoch,
                  "cost: ", self.costs[-1])
#                     ,                          "accuracy: ", self.clf_rates[-1])

epoch: 0 (33600, 784)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_9' with dtype float and shape [?,10]
	 [[node Placeholder_9 (defined at <ipython-input-24-edc94af81e18>:27) ]]

Original stack trace for 'Placeholder_9':
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home

epoch: 0 (33600, 784)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype float and shape [?,10]
	 [[node Placeholder_5 (defined at <ipython-input-19-9c0a22683632>:24) ]]

Original stack trace for 'Placeholder_5':
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/caio/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/caio/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home